In [ ]:
import importlib
import ifera
import torch

importlib.reload(ifera)

config = ifera.ConfigManager()
# instrument = config.get_config('IBKR', 'NIY:1m')

# ifera.check_s3_file_exists(S3_BUCKET, ifera.make_s3_key(instrument, True))

# df_raw = ifera.load_data(raw=True, instrument=instrument, zipfile=True)

# df = ifera.load_data(raw=False, instrument=instrument, zipfile=True, reset=True)

# instrument = config.get_config('IBKR', 'HE:1m')
# t_old = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
# t_new = ifera.load_data_tensor(instrument=instrument, reset=True, device=torch.device('cuda:0'), dtype=torch.float32)

# t_new.isclose(t_old).all()

# instrument = config.get_config('IBKR', 'NN:1m')
# t_nn = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

instrument = config.get_config('IBKR', 'ES:1m')
t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

t.shape

In [ ]:
t.shape
t=t[]

In [ ]:
import datetime as dt

instrument = config.get_config('IBKR', 'CL:1m')
t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
t[-60 * 8 + 30], dt.date.fromordinal(int(t[-60 * 8 + 30, 0].item())), dt.timedelta(seconds=t[-60 * 8 + 30, 1].item())

In [ ]:
df = ifera.load_data(raw=True, instrument=instrument, zipfile=True)

In [ ]:
# df[dt.datetime(2025, 1, 13, 9, 30)]
import pandas as pd

instrument = config.get_config('IBKR', 'NIY:1m')

df = ifera.load_data(raw=True, instrument=instrument, zipfile=True)
t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

print(df.loc[dt.datetime(2025, 1, 10, 9, 30)])
idx = -60 * 8 + 30 - 23 * 60 
s = dt.timedelta(seconds=t[idx, 1].item()).seconds
f"Date: {dt.date.fromordinal(int(t[idx, 0].item()))}, Time: {s//3600}:{(s%3600)//60}:{s%60:02d}, Open: {t[idx, 4].item():.2f}, High: {t[idx, 5].item():.2f}, Low: {t[idx, 6].item():.2f}, Close: {t[idx, 7].item():.2f}, Volume: {t[idx, 8].item():.0f}"



In [ ]:
df.tail()
df.loc[dt.datetime(2025, 1, 10, 9, 30)]

In [ ]:
import boto3

s3 = boto3.client('s3')

response = s3.list_objects(Bucket="kibotdata", Prefix='futures/1m/')
for obj in sorted(response.get('Contents', []), key=lambda x: x['Size'], reverse=True):
    print(obj['Key'], obj['Size'] // 1024**2)


In [ ]:
import ifera

config = ifera.ConfigManager()
broker = config.get_broker_config('IBKR')

for instrument in broker.instruments:
    instrument = config.get_config('IBKR', f"{instrument}:1m")
    df = ifera.load_data(raw=False, instrument=instrument, zipfile=True)



In [ ]:
import ifera
import torch
from einops import rearrange

config = ifera.ConfigManager()
broker = config.get_broker_config('IBKR')

for key in config.instruments_data.keys():
    base_inst = config.get_base_instrument_config(key)
    if base_inst.symbol not in broker.instruments:
        continue
    instrument = config.get_config('IBKR', key)
    t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
    t = rearrange(t, '(d t) c -> d t c', t = instrument.total_steps)
    print(instrument.symbol, t.shape)
    break

In [ ]:
data = t[:, :, 4:]

In [ ]:
import torch
import ifera
channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

# Relative True Range: max(high, prev_close) / min(low, prev_close), and simple high/low ratio for the first bar
rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, 0] = data[:, 0, channels["High"]] / data[:, 0, channels["Low"]] - 1.0
rtr[:, 1:] = torch.max(data[:, 1:, channels["High"]], data[:, :-1, channels["Close"]]) / torch.min(data[:, 1:, channels["Low"]], data[:, :-1, channels["Close"]]) - 1.0

# Where volume is zero set rtr to be the same as the previous value

In [ ]:
from einops import rearrange, repeat

channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

volume = data[:, :, channels["Volume"]].to(torch.int32)

# Find first non-zero volume each day
first_non_zero = torch.argmax((volume > 0).to(torch.int8), dim=1)

rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, first_non_zero] = data[:, first_non_zero, channels["High"]] / data[:, first_non_zero, channels["Low"]] - 1.0

mask = volume != 0
vdata = data[mask, :]
vrtr = rtr[mask]

raw_rtr = torch.max(vdata[1:, channels["High"]], vdata[:-1, channels["Close"]]) / torch.min(vdata[1:, channels["Low"]], vdata[:-1, channels["Close"]]) - 1.0
vrtr[1:] = torch.where(vrtr[1:] == 0, raw_rtr, vrtr[1:])

artr = ifera.ema_slow(vrtr, 1/14)

artr

In [ ]:
import torch

t = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], dtype=torch.float32)
window = 3

n = min(t.shape[-1], window)
tail = t.unfold(dimension=-1, size=n, step=1).mean(dim=-1)

head_nom = torch.cumsum(t[..., :n-1], dim=-1)
head_denom = torch.arange(1, n, device=t.device, dtype=t.dtype)
head = head_nom / head_denom

sma = torch.cat([head, tail], dim=-1)
sma


In [ ]:
import torch
import ifera.series as s
import ifera.masked_series as ms
from torch.masked import MaskedTensor, masked_tensor
from einops import repeat, rearrange

t = torch.tensor([float('nan'), 1.0, 2.0, float('nan'), float('nan'), 3.0, 4.0, 5.0, float('nan'), float('nan')], dtype=torch.float32)
t = repeat(t, 't -> t c', c = 5).clone()
t[:, 1] = t[:, 0] + 1.0 + (0.4 * torch.randn_like(t[:, 0])).round(decimals=2)
t[:, 2] = t[:, 0] - 1.0 + (0.4 * torch.randn_like(t[:, 0])).round(decimals=2)
t[:, 3] = t[:, 0] + 0.0 + (0.4 * torch.randn_like(t[:, 0])).round(decimals=2)
t[:, 4] = t[:,0] * 1000
t[:, 0:4] += 10.0
mask = ~torch.isnan(t)
t = repeat(t, 't c -> d t c', d = 2)
mask = repeat(mask, 't c -> d t c', d = 2)

mt = masked_tensor(t, mask)

#ms.masked_rtr(mt), t[1, 1]/t[1,2] -1, torch.max(t[5, 1], t[2, 3]) / torch.min(t[5, 2], t[2, 3]) - 1.0

# t_cn = rearrange(t, 'd t c -> d c t')
# mask_cn = rearrange(mask, 'd t c -> d c t')
# ct = ms.compress_tensor(t_cn, mask_cn)
# ct = rearrange(ct, 'd c t -> d t c').nan_to_num(nan=1.0)

ms.masked_rtr(mt), ms.masked_artr(mt, 0.2, True)